In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM, 
                          AutoTokenizer, 
                          GenerationConfig, 
                          TrainingArguments, 
                          Trainer)
import torch
import time
import os
import evaluate
import pandas as pd
import numpy as np
from math import ceil

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-08-19 18:49:53.071751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 18:49:54.490865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-19 18:49:54.490982: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  
torch.cuda.device_count()

1

In [3]:
# # load peft model
# from peft import PeftModel, PeftConfig

# peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# peft_model = PeftModel.from_pretrained(peft_model_base, 
#                                        './checkpoint/', 
#                                        torch_dtype=torch.bfloat16,
#                                        is_trainable=True).to("cuda")

In [4]:
# # load original model
# model_name='google/flan-t5-base'
# # model_name = "./checkpoint/"

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
class PeftModel:
    @staticmethod
    def load_base_model(model_path="google/flan-t5-base"):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_path, torch_dtype=torch.bfloat16
        ).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        return model, tokenizer

    @staticmethod
    def load_from_peft_adapter(
        base_model_path, peft_model_path, train=False
    ):
        model, tokenizer = self.load_base_model(base_model_path)
        model = PeftModel.from_pretrained(
            model, peft_model_path, torch_dtype=torch.bfloat16, is_trainable=False
        ).to(device)

        model = model.merge_and_unload()

        if train:
            for param in model.parameters():
                param.requires_grad = True

        # merge the adapter to the main model
        return model, tokenizer

    @staticmethod
    def save_peft_adapter(model, tokenizer, model_path):
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

    @staticmethod
    def merge_peft_and_save(model, tokenizer, model_path):
        model = model.merge_and_unload()
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

In [6]:
# load original model
name='google/flan-t5-base'
model, tokenizer = PeftModel.load_base_model(model_path=name)

<a name='1.2'></a>
### 1.2 - Load Dataset and LLM

In [7]:
# load and aggregate raw data
import os
import json

# Specify the folder path containing the JSON files
folder_path = './data'

# Initialize an empty list to aggregate the data
data = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        # Read and parse JSON data from the file
        with open(file_path, 'r') as json_file:
            file_data = json.load(json_file)
            
            # Assuming each JSON file contains a list of dictionaries
            if isinstance(file_data, list):
                data.extend(file_data)

# curate data

In [8]:
import random

data = [news for news in data if news["full_text"] != "" and "JavaScript is not available" not in news["full_text"]]
random.shuffle(data)
len(data)

44131

In [9]:
data[0]

{'title': "Supreme Court rejects Google's plea against record ₹1,337 crore fine",
 'summary': "The Supreme Court on Thursday refused to entertain Google's plea against the record ₹1,337 crore in an antitrust case for abusing its dominant position in Android. The apex court, however, gave Google a week's extension to deposit 10% of the penalty. Earlier, the Competition Commission of India asked Google to change restrictions imposed on smartphone makers related to pre-installing apps.",
 'link': 'https://www.hindustantimes.com/technology/supreme-court-rejects-google-s-plea-against-anti-trust-body-s-rs-1-337-cr-fine-101674128214718-amp.html?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts',
 'image_link': 'https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2023/01_jan/19_thu/img_1674134417519_29.jpg?',
 'source': 'inshorts',
 'full_text': "The Supreme Court on Thursday refused to entertain the plea of US tech giant Google against an order of the National Company

In [10]:
import random
from tqdm import tqdm
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split


class TextDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.inputs = []
        self.labels = []
        
        for news in data:   
            input_prompt, label = self._get_summary_prompt(news)
            self.inputs.append(input_prompt)
            self.labels.append(label)
            
            input_prompt, label = self._get_title_prompt(news)
            self.inputs.append(input_prompt)
            self.labels.append(label)
            
        """
        Combine the lists using zip
        Shuffle the combined list
        Unpack the shuffled pairs back into separate lists
        And then tokenize
        """
        combined = list(zip(self.inputs, self.labels))
        random.shuffle(combined)
        self.inputs, self.labels = zip(*combined)

        # tokenize
        self.inputs = tokenizer(self.inputs, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt").input_ids

        self.labels = tokenizer(self.labels, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt").input_ids
            
    def __len__(self): 
        return len(self.inputs)

    def __getitem__(self, idx): 
        return self.inputs[idx], self.labels[idx]
    
    @staticmethod
    def _get_summary_prompt(example):
        # word count round off
        multiple = 25
        word_count = len(example["summary"].split())
        word_count = int(round(word_count / multiple)) * multiple
        
        start_prompt = f'Summarize this news article in {word_count} words.\n\n'
        end_prompt = '\n\nSummary: '

        prompt = start_prompt + example["full_text"] + end_prompt

        return prompt, example["summary"]
    
    @staticmethod
    def _get_title_prompt(example):
        # word count round off
        multiple = 5
        word_count = len(example["title"].split())
        word_count = int(ceil(word_count / multiple)) * multiple
        
        start_prompt = f'Give a title to the given news article in not more than {word_count} words.\n\n'
        mid_prompt = '\n\nSummary: '
        end_prompt = '\n\nTitle: '

        prompt = start_prompt + example["full_text"] + mid_prompt + example["summary"] + end_prompt
        return prompt, example["title"]

In [11]:
train_data = TextDataset(data, tokenizer)
# test_data = TextDataset(data, tokenizer)

In [12]:
train_data[0]

(tensor([12198,  1635,  1737,    48,  1506,  1108,    16,   943,  1234,     5,
          8642, 13877,    29, 18602,    15,    17,    18, 10477,  2504,  1859,
          2618, 21269,  2091,     9,     3, 24266,  6697, 26205,    52,  2659,
          1047,    71, 19271,    21,   112,     3, 25434,   161, 29136,    11,
         29395,     3, 27759,    44,     8,  3134,     7,     5,    37,   146,
            32,    33, 24558,    21,     8,   638, 17551,    32,  5500, 10109,
             7,    16,     8, 13877,  6552,  3815,    41,   434,  5329,   201,
            28, 21269,  2091,     9, 14268,    53,     8,   596,     5,  2659,
          1047,    19,  1083,     8,   163, 15839,    16,     8,   420,   662,
            30,     8,     3, 24291, 16890,    16,    66,   386, 10874,     5,
           216,    92,  1310,  2237,  6697,    12,     3,     9,     3, 19423,
          2300,   939,   872, 14710,    16,  8642, 13877,     3,  3565,     3,
             9,     3, 28595,    15,   939,    28,  

In [13]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        
        if param.requires_grad:
            trainable_model_params += param.numel()
    
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [14]:
# # FULL MODEL TRAINING
# EPOCH = 1

# training_args = TrainingArguments(
#                                   save_steps=5000,
#                                   warmup_steps=10,
#                                   logging_steps=100,
#                                   weight_decay=0.01,
#                                   num_train_epochs=EPOCH,
#                                   logging_dir='./logs',
#                                   output_dir='./checkpoint',
#                                   per_device_eval_batch_size=32,
#                                   per_device_train_batch_size=32)

# Trainer(model=model,
#         args=training_args,
#         eval_dataset=test_data,
#         train_dataset=train_data,
#         data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 
#                                     'labels': torch.stack([f[1] for f in data])}).train()

In [ ]:
# PEFT MODEL TRAINING
from peft import LoraConfig, get_peft_model, TaskType


EPOCH = 5

lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))


peft_training_args = TrainingArguments(
                                  # save_steps=5000,
                                  save_strategy="no",
                                  warmup_steps=10,
                                  logging_steps=1000,
                                  weight_decay=0.01,
                                  num_train_epochs=EPOCH,
                                  logging_dir='./logs',
                                  output_dir='./checkpoint',
                                  learning_rate=0.0001,
                                  auto_find_batch_size=True)
    
peft_trainer = Trainer(
                model=peft_model,
                args=peft_training_args,
                train_dataset=train_data,
                data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 
                                            'labels': torch.stack([f[1] for f in data])})

peft_trainer.train()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


trainable model parameters: 7077888
all model parameters: 254655744
percentage of trainable model parameters: 2.78%


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,3.106900
2000,0.256400
3000,0.237700
4000,0.224600
5000,0.223300
6000,0.216200
7000,0.210600
8000,0.274000
9000,0.208300
10000,0.205500


# save model

In [ ]:
# merge peft with main model
# and save the model
model_path = "./checkpoint/"
PeftModel.merge_peft_and_save(model=peft_model, tokenizer=tokenizer, model_path)

# load pretrained flan t5 

In [16]:
# load original model
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to("cuda")
original_tokenizer = AutoTokenizer.from_pretrained(model_name)

# infer

In [52]:
dash_line = '-'.join('' for x in range(100))

index = 5678
news = data[index]
full_text = news['full_text']
baseline_human_summary = news['summary']

# word count round off
multiple = 25
word_count = len(baseline_human_summary.split())
word_count = int(round(word_count / multiple)) * multiple

start_prompt = f'Summarize this news article in {word_count} words.\n\n'
end_prompt = '\n\nSummary: '

prompt = start_prompt + full_text + end_prompt

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'FULL TEXT:\n{full_text}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


# EVALUATE
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=[original_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=[peft_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('INSTRUCT MODEL:')
print(instruct_model_results)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
An old video showing the Trinity Test, the first atomic bomb test, in US in 1945 has resurfaced on social media. Several people compared the footage with the scene in Christopher Nolan's film 'Oppenheimer'. "Nolan portrayed the explosion extremely accurate," a social media user said. "Absolutely terrifying," said another user.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
A video claiming to be the actual footage of the Trinity Test has surfaced on Instagram.
---------------------------------------------------------------------------------------------------
PEFT MODEL: A video claiming to be the actual footage of the Trinity Test has surfaced on Instagram. The video shows J Robert Oppenheimer and his team prepping for the historic explosion test. Men at the site felt the heat of a desert sun. Th